In [1]:
import pandas as pd

df = pd.read_csv('../data/wiki_text_cleaned_v1.csv')

# check if the id column is unique
print(df['id'].is_unique)

# check if the id column is a primary key
print(df['id'].is_monotonic_increasing)

True
True
